In [1]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI

In [2]:
# dir(SQLDatabase)
import os, sys
sys.path.insert(1,'D:\Github\Tabular-Data-with-LLMs' )
import credentials
os.environ["OPENAI_API_KEY"] = credentials.openai

In [4]:
db = SQLDatabase.from_uri("sqlite:///data/tab_db.db")
toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0))

In [5]:
dir(db)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_all_tables',
 '_custom_table_info',
 '_engine',
 '_get_sample_rows',
 '_get_table_indexes',
 '_ignore_tables',
 '_include_tables',
 '_indexes_in_table_info',
 '_inspector',
 '_max_string_length',
 '_metadata',
 '_sample_rows_in_table_info',
 '_schema',
 '_usable_tables',
 'dialect',
 'from_databricks',
 'from_uri',
 'get_table_info',
 'get_table_info_no_throw',
 'get_table_names',
 'get_usable_table_names',
 'run',
 'run_no_throw',
 'table_info']

In [6]:
db.table_info

'\nCREATE TABLE tabular_data (\n\t"Report Time" TEXT, \n\t"Product Name" TEXT, \n\t"Sub Product" TEXT, \n\t"RegionName" TEXT, \n\t"MarketName" TEXT, \n\t"TermName" TEXT, \n\t"Value" REAL, \n\t"Unit" TEXT, \n\t"Index" INTEGER, \n\tPRIMARY KEY ("Index")\n)\n\n/*\n3 rows from tabular_data table:\nReport Time\tProduct Name\tSub Product\tRegionName\tMarketName\tTermName\tValue\tUnit\tIndex\n2023-04-28 00:00:00\tWoven Yarn\t10/1 Carded\tAsia\tPakistan\tCNF\t2.7\tUSD/Kg\t1\n2023-04-27 00:00:00\tWoven Yarn\t10/1 Carded\tAsia\tPakistan\tCNF\t2.7\tUSD/Kg\t2\n2023-04-26 00:00:00\tWoven Yarn\t10/1 Carded\tAsia\tPakistan\tCNF\t2.71\tUSD/Kg\t3\n*/'

In [7]:
agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [8]:
# Describe your query here
query = """\
Describe the tabular_data table
"""

agent_executor.run(query)



> Entering new  chain...


Action: sql_db_list_tables
Action Input: ""
Observation: tabular_data
Thought: I should query the schema of the tabular_data table
Action: sql_db_schema
Action Input: "tabular_data"
Observation: 
CREATE TABLE tabular_data (
	"Report Time" TEXT, 
	"Product Name" TEXT, 
	"Sub Product" TEXT, 
	"RegionName" TEXT, 
	"MarketName" TEXT, 
	"TermName" TEXT, 
	"Value" REAL, 
	"Unit" TEXT, 
	"Index" INTEGER, 
	PRIMARY KEY ("Index")
)

/*
3 rows from tabular_data table:
Report Time	Product Name	Sub Product	RegionName	MarketName	TermName	Value	Unit	Index
2023-04-28 00:00:00	Woven Yarn	10/1 Carded	Asia	Pakistan	CNF	2.7	USD/Kg	1
2023-04-27 00:00:00	Woven Yarn	10/1 Carded	Asia	Pakistan	CNF	2.7	USD/Kg	2
2023-04-26 00:00:00	Woven Yarn	10/1 Carded	Asia	Pakistan	CNF	2.71	USD/Kg	3
*/
Thought: I now know the final answer
Final Answer: The tabular_data table contains columns for Report Time, Product Name, Sub Product, RegionName, MarketName, TermName, Value, Unit, and Index.

> Finished chain.


'The tabular_data table contains columns for Report Time, Product Name, Sub Product, RegionName, MarketName, TermName, Value, Unit, and Index.'

In [10]:
# Describe your query here
query = """\
What is the average price of the last 2 years of Product Knitting yarn with sub product 40/1 carded?
"""

agent_executor.run(query)



> Entering new  chain...
Action: sql_db_list_tables
Action Input: ""
Observation: tabular_data
Thought: I should query the schema of the tabular_data table to see what columns I can use.
Action: sql_db_schema
Action Input: tabular_data
Observation: 
CREATE TABLE tabular_data (
	"Report Time" TEXT, 
	"Product Name" TEXT, 
	"Sub Product" TEXT, 
	"RegionName" TEXT, 
	"MarketName" TEXT, 
	"TermName" TEXT, 
	"Value" REAL, 
	"Unit" TEXT, 
	"Index" INTEGER, 
	PRIMARY KEY ("Index")
)

/*
3 rows from tabular_data table:
Report Time	Product Name	Sub Product	RegionName	MarketName	TermName	Value	Unit	Index
2023-04-28 00:00:00	Woven Yarn	10/1 Carded	Asia	Pakistan	CNF	2.7	USD/Kg	1
2023-04-27 00:00:00	Woven Yarn	10/1 Carded	Asia	Pakistan	CNF	2.7	USD/Kg	2
2023-04-26 00:00:00	Woven Yarn	10/1 Carded	Asia	Pakistan	CNF	2.71	USD/Kg	3
*/
Thought: I should query the tabular_data table for the average price of the last 2 years of Product Knitting yarn with sub product 40/1 carded.
Action: sql_db_query
Actio

'The average price of the last 2 years of Product Knitting yarn with sub product 40/1 carded is 9252.599747722485.'

In [11]:
# Describe your query here
query = """\
What is the average price of the year 2023 of Product Knitting yarn with sub product 40/1 carded for each Unit type?
"""

agent_executor.run(query)



> Entering new  chain...
Action: sql_db_list_tables
Action Input: ""
Observation: tabular_data
Thought: I should query the schema of the tabular_data table to see what columns I can use.
Action: sql_db_schema
Action Input: tabular_data
Observation: 
CREATE TABLE tabular_data (
	"Report Time" TEXT, 
	"Product Name" TEXT, 
	"Sub Product" TEXT, 
	"RegionName" TEXT, 
	"MarketName" TEXT, 
	"TermName" TEXT, 
	"Value" REAL, 
	"Unit" TEXT, 
	"Index" INTEGER, 
	PRIMARY KEY ("Index")
)

/*
3 rows from tabular_data table:
Report Time	Product Name	Sub Product	RegionName	MarketName	TermName	Value	Unit	Index
2023-04-28 00:00:00	Woven Yarn	10/1 Carded	Asia	Pakistan	CNF	2.7	USD/Kg	1
2023-04-27 00:00:00	Woven Yarn	10/1 Carded	Asia	Pakistan	CNF	2.7	USD/Kg	2
2023-04-26 00:00:00	Woven Yarn	10/1 Carded	Asia	Pakistan	CNF	2.71	USD/Kg	3
*/
Thought: I should query the tabular_data table for the average price of Product Knitting yarn with sub product 40/1 carded in the year 2023.
Action: sql_db_query
Action I

'The average price of Product Knitting yarn with sub product 40/1 carded in the year 2023 is 8126.5807058823375 USD/Kg.'

In [12]:
# Describe your query here
query = """\
What is the average price of the year 2023 of Product Knitting yarn with sub product 40/1 carded for each Unit type?
"""

agent_executor.run(query)



> Entering new  chain...
Action: sql_db_list_tables
Action Input: ""
Observation: tabular_data
Thought: I should query the schema of the tabular_data table to see what columns I can use.
Action: sql_db_schema
Action Input: tabular_data
Observation: 
CREATE TABLE tabular_data (
	"Report Time" TEXT, 
	"Product Name" TEXT, 
	"Sub Product" TEXT, 
	"RegionName" TEXT, 
	"MarketName" TEXT, 
	"TermName" TEXT, 
	"Value" REAL, 
	"Unit" TEXT, 
	"Index" INTEGER, 
	PRIMARY KEY ("Index")
)

/*
3 rows from tabular_data table:
Report Time	Product Name	Sub Product	RegionName	MarketName	TermName	Value	Unit	Index
2023-04-28 00:00:00	Woven Yarn	10/1 Carded	Asia	Pakistan	CNF	2.7	USD/Kg	1
2023-04-27 00:00:00	Woven Yarn	10/1 Carded	Asia	Pakistan	CNF	2.7	USD/Kg	2
2023-04-26 00:00:00	Woven Yarn	10/1 Carded	Asia	Pakistan	CNF	2.71	USD/Kg	3
*/
Thought: I should query the tabular_data table for the average price of Product Knitting yarn with sub product 40/1 carded for each Unit type in the year 2023.
Action: sq

'The average price of Product Knitting yarn with sub product 40/1 carded for each Unit type in the year 2023 is 24110.49 CNY/mt, 265.59 INR/Kg, and 3.66 USD/Kg.'

In [19]:
# Describe your query here
query = """\
What is the average price from 1st jan 2020 to 1st july 2020 of sub product 40/1 carded?
"""

agent_executor.run(query)



> Entering new  chain...
Action: sql_db_list_tables
Action Input: ""
Observation: tabular_data
Thought: I should query the schema of the tabular_data table to see what columns I can use.
Action: sql_db_schema
Action Input: "tabular_data"
Observation: 
CREATE TABLE tabular_data (
	"Report Time" TEXT, 
	"Product Name" TEXT, 
	"Sub Product" TEXT, 
	"RegionName" TEXT, 
	"MarketName" TEXT, 
	"TermName" TEXT, 
	"Value" REAL, 
	"Unit" TEXT, 
	"Index" INTEGER, 
	PRIMARY KEY ("Index")
)

/*
3 rows from tabular_data table:
Report Time	Product Name	Sub Product	RegionName	MarketName	TermName	Value	Unit	Index
2023-04-28 00:00:00	Woven Yarn	10/1 Carded	Asia	Pakistan	CNF	2.7	USD/Kg	1
2023-04-27 00:00:00	Woven Yarn	10/1 Carded	Asia	Pakistan	CNF	2.7	USD/Kg	2
2023-04-26 00:00:00	Woven Yarn	10/1 Carded	Asia	Pakistan	CNF	2.71	USD/Kg	3
*/
Thought: I should query the tabular_data table for the average price of sub product 40/1 carded between 1st jan 2020 and 30th june 2020.
Action: sql_db_query
Action Inp

'The average price of sub product 40/1 carded between 1st jan 2020 and 30th june 2020 is 140.57953125000006.'

In [20]:
# Describe your query here
query = """\
What is price trend of product Cotton yarn Woven yarn last 4 quarters of the Indian market?
"""

agent_executor.run(query)



> Entering new  chain...
Action: sql_db_list_tables
Action Input: 
Observation: tabular_data
Thought: I should query the schema of the tabular_data table to see what columns I can use.
Action: sql_db_schema
Action Input: tabular_data
Observation: 
CREATE TABLE tabular_data (
	"Report Time" TEXT, 
	"Product Name" TEXT, 
	"Sub Product" TEXT, 
	"RegionName" TEXT, 
	"MarketName" TEXT, 
	"TermName" TEXT, 
	"Value" REAL, 
	"Unit" TEXT, 
	"Index" INTEGER, 
	PRIMARY KEY ("Index")
)

/*
3 rows from tabular_data table:
Report Time	Product Name	Sub Product	RegionName	MarketName	TermName	Value	Unit	Index
2023-04-28 00:00:00	Woven Yarn	10/1 Carded	Asia	Pakistan	CNF	2.7	USD/Kg	1
2023-04-27 00:00:00	Woven Yarn	10/1 Carded	Asia	Pakistan	CNF	2.7	USD/Kg	2
2023-04-26 00:00:00	Woven Yarn	10/1 Carded	Asia	Pakistan	CNF	2.71	USD/Kg	3
*/
Thought: I should query the tabular_data table for the price trend of product Cotton yarn Woven yarn last 4 quarters of the Indian market.
Action: sql_db_query
Action Input

'The price trend of product Cotton yarn Woven yarn last 4 quarters of the Indian market is [list of prices].'